# Set up Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()
import openai, os
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_key = os.environ["OPENAI_API_KEY"]

print("OpenAI Endpoint:", openai.api_base)
print("MLFlow Tracking URI: ", os.getenv("MLFLOW_TRACKING_URI"))

OpenAI Endpoint: https://aoai.openai.azure.com/
MLFlow Tracking URI:  azureml://eastus2.api.azureml.ms/mlflow/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/ray/providers/Microsoft.MachineLearningServices/workspaces/ray


## Chat Model

In [42]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

reply = chat([SystemMessage(content="You are a friendly chat agent."),
              HumanMessage(content="Hello, how are you?")])

print(reply.content)

json_call: {'module': 'langchain.chat_models.openai', 'name': '_generate', 'args': (AzureChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa0217edfc0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='', request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None, deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_base='', openai_api_version='2023-03-15-preview'), [SystemMessage(content='You are a friendly chat agent.', additional_kwargs={}), HumanMessage(content='Hello, how are you?', additional_kwargs={})]), 'kwargs': {'stop': None}}
json_result: generations=[ChatGeneration(text="Hello! I'm just an AI language model, so I don't have feelings, but I am programmed to assist and have a conversation with you. How can I help you today?", generation_info=None, message=AIMessage(content="Hello! I'm just an A

## Completion Model

### Zero Shot

In [7]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)
text="""
Classify the following news headline into 1 of the following categories: Business, Tech, Politics, Sport, Entertainment

Headline: Major Retailer Announces Plans to Close Over 100 Stores
Category:
"""

print(llm(text))

Business


### Few Shot

In [6]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text="""
Classify the following news headline into 1 of the following categories: Business, Tech, Politics, Sport, Entertainment

Headline 1: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet’s most beloved cooking guru has a buzzy new book and a fresh new perspective
Category: Entertainment

Headline 2: Major Retailer Announces Plans to Close Over 100 Stores
Category:
"""

print(llm(text))

Business


### Chain of Reasoning

In [18]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text="""
what is the anwer to this equation: 3x - 7 = -2x + 5
"""

print(llm(text))


The answer is x = 4.


In [17]:
from sympy import *
x = symbols('x')
solve(3*x - 7 + 2*x - 5, x)


[12/5]

In [16]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text="""
solve the following algebraic equation step-by-step and explain you reasoning at each step: 3x - 7 = -2x + 5
"""

print(llm(text))


Step 1: Add 7 to both sides of the equation.
3x - 7 + 7 = -2x + 5 + 7

Step 2: Simplify the left side of the equation.
3x = -2x + 12

Step 3: Subtract -2x from both sides of the equation.
3x - (-2x) = -2x + 12 - (-2x)

Step 4: Simplify the left side of the equation.
5x = 12

Step 5: Divide both sides of the equation by 5.
x = 12/5

Step 6: Simplify the right side of the equation.
x = 2.4


# Chains

https://python.langchain.com/en/latest/modules/chains/getting_started.html

In [31]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("organic snacks"))

json_call: {'module': 'langchain.chat_models.openai', 'name': '_generate', 'args': (AzureChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa0217edfc0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='', request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None, deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_base='', openai_api_version='2023-03-15-preview'), [HumanMessage(content='What is a good name for a company that makes organic snacks?', additional_kwargs={})]), 'kwargs': {'stop': None}}
json_result: generations=[ChatGeneration(text='Here are 5 potential names: \n\n1. EarthBites \n2. Purely Nosh \n3. Wholesome Munchies \n4. FreshFusion \n5. NatureNibbles', generation_info=None, message=AIMessage(content='Here are 5 potential names: \n\n1. EarthBites \n2. Purely Nosh \n3. Wholesome Munchies \n

# Patch Langchain

In [5]:
from src.langchain.patch import patch_langchain

patch_langchain()

# Agent

In [43]:
import mlflow
mlflow.end_run()

In [44]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI, AzureOpenAI
from langchain.schema import HumanMessage
import openai, os

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("I have 1000 dollars. given today's governement bond rate, how much money would I have in 2 years if I invested them in 2 year government bonds?")




> Entering new AgentExecutor chain...
json_call: {'module': 'langchain.llms.openai', 'name': '_generate', 'args': (AzureOpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa0217edfc0>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.0, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, deployment_name='text-davinci-003'), ["Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think ab

'In two years, I will have 1083.26 dollars if I invest 1000 dollars in 2 year government bonds.'

# "Retrieval Augmented Generation"-Style Chain

In [21]:
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever, Document
from typing import List
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

class CognitiveSearchRetriever(BaseRetriever):
    def __init__(self, endpoint: str, index_name: str, searchkey: str, top: int = 3):
        self.endpoint = endpoint
        self.index_name = index_name
        self.searchkey = searchkey
        self.top = top
        self.client = SearchClient(endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(searchkey))

    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = []
        for i in self.client.search(query, top=self.top):
            docs.append(Document(page_content=i['content'], metadata={"sourcefile": i['sourcefile']}))
        return docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        pass

In [22]:
cog_search = CognitiveSearchRetriever(endpoint=os.getenv("COG_SEARCH_ENDPOINT"), index_name="amldocs", searchkey=os.environ["COG_SEARCH_KEY"])


In [23]:
cog_search.get_relevant_documents("pipelines")

[Document(page_content='   > You can publish multiple pipelines to a single endpoint. Each pipeline in a given endpoint is given a version number, which you can specify when you call the pipeline endpoint.\n\n1. Select **Publish**.\n\n## Retrain your model\n\nNow that you have a published training pipeline, you can use it to retrain your model on new data. You can submit jobs from a pipeline endpoint from the studio workspace or programmatically.\n\n### Submit jobs by using the studio portal\n\nUse the following steps to submit a parameterized pipeline endpoint job from the studio portal:\n\n1. Go to the **Endpoints** page in your studio workspace.\n1. Select the **Pipeline endpoints** tab. Then, select your pipeline endpoint.\n1. Select the **Published pipelines** tab. Then, select the pipeline version that you want to run.\n1. Select **Submit**.\n1. In the setup dialog box, you can specify the parameters values for the job. For this example, update the data path to train your model u

In [24]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

qa = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff",
                                    retriever=cog_search)

In [25]:
qa("What is a pipeline?")

json_call: {'module': 'langchain.chat_models.openai', 'name': '_generate', 'args': (AzureChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa0217edfc0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='', request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None, deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_base='', openai_api_version='2023-03-15-preview'), [SystemMessage(content='Use the following pieces of context to answer the users question. \nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n----------------\n\n# MLOps: Model management, deployment, and monitoring with Azure Machine Learning\n\n[!INCLUDE [dev v2](../../includes/machine-learning-dev-v2.md)]\n\n> [!div class="op_single_selector" title1="Select the version of Azure Machine Learning 

{'query': 'What is a pipeline?',
 'result': 'A pipeline in machine learning is a collection of repeatable and reusable steps used for data preparation, model training, and evaluation. Machine learning pipelines run automatically to train, tune, and deploy models. These pipelines enable easier management of the workflow and helps in creating reproducible results.'}